In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.INFO)
from common import data, fit
from common.util import download_file
import mxnet as mx
import numpy as np
from mxnet.symbol import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def download_cifar100():
    data_dir="/efs/data/cifar-100-mxnet"
    #data_dir="/data/cifar-100-mxnet"
    fnames = (os.path.join(data_dir, "train.rec"),
              os.path.join(data_dir, "test.rec"))
    return fnames

In [4]:
(train_fname, val_fname) = download_cifar100()

# parse args
parser = argparse.ArgumentParser(description="train cifar100",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
fit.add_fit_args(parser)
data.add_data_args(parser)
data.add_data_aug_args(parser)
data.set_data_aug_level(parser, 2)

num_unsup_examples = 50000
subset_factor = 5

parser.set_defaults(
    # data
    data_train     = train_fname,
    data_val       = val_fname,
    num_classes    = 100,
    image_shape    = '3,28,28',
    prefix         = 'cifar_b250_fine',
    pad_size       = 2,
    # train
    batch_size     = 250,  # todo currently has to devide 'num_validation_samples'
    num_epochs     = 380,
    lr_step_epochs = '180,270',  # this setting should converge to a good result
    gpus           = '0,1,2,3,4,5,6,7',
    loss_gpu       = 0,
    disp_batches   = 20,
    num_examples   = num_unsup_examples / subset_factor, 
    wd             = 1e-4,
    lr             = .001,
    lr_factor      = .33,
    nembeddings    = 256,
    optimizer      = 'adam'
)

args = parser.parse_args("")

unsup_multiplier = 2
labeled_per_class = 500 / subset_factor

sample_seed = 47
val_interval = 1
save_interval = 10

In [5]:
from common.multi_iterator import Multi_iterator
from common.data import get_partial_rec_iter

(train_sup, val) = get_partial_rec_iter(args, get_val=True, devide_by=subset_factor, shuffle=True)
(train_unsup, _) = get_partial_rec_iter(args, get_val=False, devide_by=1, shuffle=True)
    
num_sup_examples = labeled_per_class * args.num_classes
train = Multi_iterator(train_sup, train_unsup, unsup_multiplier, num_unsup_examples, num_sup_examples)

4  times more unsup data than sup data


In [6]:
from symbols import inception_cifar as base_net
from common.lba import compute_semisup_loss, logit_loss
from common.wrapper_module import get_embedding_shapes

# kvstore
kv = mx.kvstore.create(args.kv_store)

t_nb = args.batch_size * unsup_multiplier
#initializer = mx.init.Xavier(rnd_type='gaussian', factor_type="avg", magnitude=2.34)
initializer = mx.init.Uniform(0.01)

lr, lr_scheduler = fit._get_lr_scheduler(args, kv)

def buildEmbeddingModule(arg_p=None, aux_p=None):
    dataSup = mx.symbol.Variable(name="dataSup")
    dataUnsup = []
    for i in range(unsup_multiplier):
        dataUnsup.append(Variable(name="dataUnsup"+str(i)))

    # concat data, feed both through the network
    # then split it up again
    data = concat(dataSup, *dataUnsup, dim=0)

    embeddings = base_net.build_embeddings(data, nembeddings=args.nembeddings)
    splitted = split(embeddings, num_outputs=(unsup_multiplier+1), axis=0, name='split')
    
    # devices for training
    devs = mx.cpu() if args.gpus is None or args.gpus is '' else [
        mx.gpu(int(i)) for i in args.gpus.split(',')]

    data_names = ['dataSup'] + ['dataUnsup'+str(i) for i in range(unsup_multiplier)]
        
    # create model
    model = mx.mod.Module(
        context       = devs,
        symbol        = splitted,
        data_names    = data_names,
        label_names   = None)
    
    model.bind(data_shapes=train.provide_data)         
    model.init_params(initializer, arg_p, aux_p)    
    model.init_optimizer(optimizer=args.optimizer, optimizer_params=(
       ('learning_rate', lr), 
       ('wd', 1e-4),
       #('momentum', args.mom),
       ('rescale_grad', 0.005),
       ('lr_scheduler', lr_scheduler)))

    return model


def buildSupLossModule(arg_p=None, aux_p=None):
    supEmbeddings = Variable(name="embeddings_sup")
    labelSup = mx.symbol.Variable(name='labelSup')
    overall_loss = []
    
    if unsup_multiplier >= 1:
        unsupEmbeddings = []
        for i in range(unsup_multiplier):
            unsupEmbeddings.append(Variable(name="embeddings_unsup"+str(i)))
    
        unsupEmbeddings = concat(*unsupEmbeddings, dim=0)
            
        (walker_loss, visit_loss) = compute_semisup_loss(supEmbeddings, unsupEmbeddings, labelSup, t_nb, 
                                                     walker_weight=1.0, visit_weight=0.5)
        #overall_loss = [walker_loss, visit_loss]
        
    overall_loss = [logit_loss(supEmbeddings, labelSup, args.num_classes)] + overall_loss
    
    # todo maybe use gpu
    devs = mx.cpu() if args.loss_gpu is None or args.loss_gpu is '' else mx.gpu(args.loss_gpu)

    # create module
    model = mx.mod.Module(
        context = devs,
        symbol  = Group(overall_loss),
        data_names = ['embeddings_sup'],#+['embeddings_unsup'+str(i) for i in range(unsup_multiplier)],
        label_names = ['labelSup'])
    
    # allocate memory by given the input data and label shapes
    model.bind(data_shapes=[get_embedding_shapes(args.batch_size, args.nembeddings, unsup_multiplier)[0]], 
               label_shapes=train.provide_label,
               inputs_need_grad=True)
            
    model.init_params(initializer, arg_p, aux_p)    
    model.init_optimizer(optimizer=args.optimizer, optimizer_params=(
        ('learning_rate', lr), 
        ('rescale_grad', 0.005),
        #('momentum', args.mom),
        ('wd', 1e-4),
        ('lr_scheduler', lr_scheduler)))
    
    return model


def buildLossModule(arg_p=None, aux_p=None):
    supEmbeddings = Variable(name="embeddings_sup")
    labelSup = mx.symbol.Variable(name='labelSup')
    overall_loss = []
    
    if unsup_multiplier >= 1:
        unsupEmbeddings = []
        for i in range(unsup_multiplier):
            unsupEmbeddings.append(Variable(name="embeddings_unsup"+str(i)))
    
        unsupEmbeddings = concat(*unsupEmbeddings, dim=0)
            
        (walker_loss, visit_loss) = compute_semisup_loss(supEmbeddings, unsupEmbeddings, labelSup, t_nb, 
                                                     walker_weight=0.8, visit_weight=0.5)
        overall_loss = [walker_loss, visit_loss]
        
    overall_loss = [logit_loss(supEmbeddings, labelSup, args.num_classes)] + overall_loss
    
    # todo maybe use gpu
    devs = mx.cpu() if args.loss_gpu is None or args.loss_gpu is '' else mx.gpu(args.loss_gpu)

    # create module
    model = mx.mod.Module(
        context = devs,
        symbol  = Group(overall_loss),
        data_names = ['embeddings_sup']+['embeddings_unsup'+str(i) for i in range(unsup_multiplier)],
        label_names = ['labelSup'])
    
    # allocate memory by given the input data and label shapes
    model.bind(data_shapes=get_embedding_shapes(args.batch_size, args.nembeddings, unsup_multiplier), 
               label_shapes=train.provide_label,
               inputs_need_grad=True)
            
    model.init_params(initializer, arg_p, aux_p)    
    model.init_optimizer(optimizer=args.optimizer, optimizer_params=(
        ('learning_rate', lr), 
        ('rescale_grad', 0.005),
        #('momentum', args.mom),
        ('wd', 1e-4),
        ('lr_scheduler', lr_scheduler)))
    
    return model

[3600.0]


In [9]:
from common.wrapper_module import WrapperModule
#eval_metrics = Multi_Accuracy(num= 3 if unsup_multiplier >= 1 else 1)
                    
def fit_model(args, embeddingModule, lossModule, data, **kwargs):
    """
    train a model
    args : argparse returns
    data_loader : function that returns the train and val data iterators
    """
    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    logging.basicConfig(level=logging.DEBUG, format=head)
    logging.info('start with arguments %s', args)
    
    # data iterators
    (train, val) = data

    # load model
    if 'arg_params' in kwargs and 'aux_params' in kwargs:
        arg_params = kwargs['arg_params']
        aux_params = kwargs['aux_params']
    else: 
        sym, arg_params, aux_params = fit._load_model(args, kv.rank)
        
    # save model
    checkpoint = fit._save_model(args, kv.rank)

    model = WrapperModule(embeddingModule, lossModule, unsup_multiplier)
    
    logf = open(args.prefix+'logs', 'w')
    def validate_model(epoch, *args_):
        if epoch % val_interval != 0: 
            return
        res = model.score(val)
        #TODO: pull this into default
        print('Epoch[%d] Validation-accuracy=%f' % (epoch,  res))
        logf.write('Epoch[%d] Validation-accuracy=%f \n' % (epoch,  res))  # python will convert \n to os.linesep
        logf.flush()

        if epoch % save_interval == 0:
            model.save_checkpoint(args.prefix, epoch)

    #monitor = mx.mon.Monitor(interval=1000, pattern='.*aba_backward.*') 
    monitor = mx.mon.Monitor(interval=1000, pattern='.*') 
    
    # callbacks that run after each batch
    batch_end_callbacks = [mx.callback.Speedometer(args.batch_size*(unsup_multiplier+1), args.disp_batches)]
    
    epoch_end_callbacks = validate_model

    # run
    model.fit(train,
        begin_epoch        = args.load_epoch if args.load_epoch else 0,
        num_epoch          = args.num_epochs,
        #eval_data          = val,
        #eval_metric        = eval_metrics,
        kvstore            = kv,
        arg_params         = arg_params,
        aux_params         = aux_params,
        batch_end_callback = batch_end_callbacks,
        epoch_end_callback = epoch_end_callbacks,
        allow_missing      = True
        #monitor            = monitor
             )
    logf.close()
    return model

In [ ]:
train.reset()

# train using a checkpoint with 20% validation accuracy, trained only supervised
#(sym, arg_p, aux_p) = mx.model.load_checkpoint('embedding_val30',25)
#embeddingModule = buildEmbeddingModule(arg_p, aux_p)      

# train from previous iteration
(arg_p_emb, aux_p_emb, arg_p_loss, aux_p_loss) = WrapperModule.load_checkpoint('cifarsemisup', 200)
embeddingModule = buildEmbeddingModule(arg_p_emb, aux_p_emb)      
lossModule = buildLossModule(arg_p_loss, aux_p_loss)
# train from scratch
#embeddingModule = buildEmbeddingModule()

#semisupervised loss module
#lossModule = buildLossModule()

#supervised loss module
#lossModule = buildSupLossModule()

m = fit_model(args, embeddingModule, lossModule, (train, val))

INFO:root:start with arguments Namespace(batch_size=250, benchmark=0, data_nthreads=4, data_train='/efs/data/cifar-100-mxnet/train.rec', data_val='/efs/data/cifar-100-mxnet/test.rec', disp_batches=20, dtype='float32', gpus='0,1,2,3,4,5,6,7', image_shape='3,28,28', kv_store='device', load_epoch=None, loss_gpu=0, lr=0.00033, lr_factor=0.33, lr_step_epochs='90', max_random_aspect_ratio=0, max_random_h=36, max_random_l=50, max_random_rotate_angle=0, max_random_s=50, max_random_scale=1, max_random_shear_ratio=0, min_random_scale=1, model_prefix=None, mom=0.9, monitor=0, nembeddings=256, network=None, num_classes=100, num_epochs=200, num_examples=10000.0, num_layers=None, optimizer='adam', pad_size=2, prefix='cifar_b250_fine', random_crop=1, random_mirror=1, rgb_mean='123.68,116.779,103.939', test_io=0, top_k=0, wd=0.0001)
INFO:root:Epoch[0] Batch [20]	Speed: 884.80 samples/sec	Train-accuracy=nan
INFO:root:Epoch[0] Train-accuracy=nan
INFO:root:Epoch[0] Time cost=35.060


Epoch[0] Validation-accuracy=0.506700


INFO:root:Saved checkpoint to "emb_cifar_b250_fine-0000.params"
INFO:root:Saved checkpoint to "loss_cifar_b250_fine-0000.params"
INFO:root:Epoch[1] Batch [20]	Speed: 865.74 samples/sec	Train-accuracy=nan
INFO:root:Epoch[1] Train-accuracy=nan
INFO:root:Epoch[1] Time cost=34.209


Epoch[1] Validation-accuracy=0.507900


INFO:root:Epoch[2] Batch [20]	Speed: 867.51 samples/sec	Train-accuracy=nan
INFO:root:Epoch[2] Train-accuracy=nan
INFO:root:Epoch[2] Time cost=34.360


...

INFO:root:Epoch[160] Batch [20]	Speed: 863.57 samples/sec	Train-accuracy=nan
INFO:root:Epoch[160] Train-accuracy=nan
INFO:root:Epoch[160] Time cost=34.387


Epoch[160] Validation-accuracy=0.537300


INFO:root:Saved checkpoint to "emb_cifar_b250_fine-0160.params"
INFO:root:Saved checkpoint to "loss_cifar_b250_fine-0160.params"
INFO:root:Epoch[161] Batch [20]	Speed: 865.29 samples/sec	Train-accuracy=nan
INFO:root:Epoch[161] Train-accuracy=nan
INFO:root:Epoch[161] Time cost=34.350


Epoch[161] Validation-accuracy=0.541300


KeyboardInterrupt: 

In [ ]:
aux_p

In [ ]:
embeddingModule.save_checkpoint('cifarsemisup200', 200)

In [ ]:
lossModule.save_checkpoint('cifarsemisup200_loss', 200)

In [ ]:
val.reset()
val.next().label[0].shape[0]

In [ ]:
class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):

        #for i in range(len(preds)):
        for i in range(1):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[0].asnumpy().astype('int32')

            #mx.metric.check_label_shapes(label, pred_label)
            
            #print((pred_label.flat == label.flat).sum())
            #print(len(pred_label.flat))

            
            self.sum_metric[i] += (pred_label.flat == label.flat).sum()
            self.num_inst[i] += len(pred_label.flat)

In [ ]:
# batch 10 train 0.389, valid: 0.40